In [ ]:
# packages needed
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# keras imports
import keras
from keras.applications import vgg16
from keras.layers import Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import Model
from keras import backend as K

# model selection
from sklearn.model_selection import train_test_split

# to get image names
import os, cv2, random
import ntpath

# random seed to make the kernel reproducible
random_seed = 123456789
np.random.seed(random_seed)

In [ ]:
# directory where we are going to work on
workdir = './data'

In [ ]:
# Create result dir
result_dir = './results' # define here the directory where your results will be saved
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

# Dataset description

**************************************************************************
Content of **AVA.txt**
**************************************************************************

* **Column 1:** Index

* **Column 2:** Image ID 

* **Columns 3 - 12:** Counts of aesthetics ratings on a scale of 1-10. Column 3 has counts of ratings of 1 and column 12 has counts of ratings of 10.

* **Columns 13 - 14:** Semantic tag IDs. There are 66 IDs ranging from 1 to 66. The file tags.txt contains the textual tag corresponding to the numerical id. Each image has between 0 and 2 tags. Images with less than 2 tags have a "0" in place of the missing tag(s).

* **Column 15:** Challenge ID. The file challenges.txt contains the name of the challenge corresponding to each ID.

## Load the data

In [ ]:
# set the correct directory
ava_dir = os.path.join(workdir, 'AVA.txt')
ava_data_pd = pd.read_csv(ava_dir, sep = ' ', header = None, names = ['Index', 'Image ID', 'Rate 1', 'Rate 2',
                                                                      'Rate 3', 'Rate 4', 'Rate 5', 'Rate 6', 
                                                                      'Rate 7', 'Rate 8', 'Rate 9','Rate 10', 
                                                                      'Sem 1', 'Sem 2', 'Challenge ID'])

# separate the labels of the image IDs
x_train = ava_data_pd.values[:,1]
y_train = ava_data_pd.values[:,2:12]

In [ ]:
# sample to see if everything has been loaded correctly
ava_data_pd.sample(5)

## Data cleaning

Once all the data is loaded, we will check if all the images are in the directory and all that are not will be removed.

In [ ]:
# define directory of training images
train_dir_images = os.path.join(workdir, 'images')

# list containing the ids of the images that are not in the directory
lost_images = []

# iterate over all images
for i in range(len(x_train)):
    file_path = os.path.join(train_dir_images, str(x_train[i]) + '.jpg')
    if not os.path.exists(file_path):
        lost_images.append(i)

# mask operations
mask = np.ones((len(x_train),), bool)
mask[lost_images] = False # set to false all lost images

# prints number of lost images and original shape
print("Total images lost: ", len(lost_images))
print("Original dataset shape: ", x_train.shape)

x_train = x_train[mask]
y_train = y_train[mask]

# resulting shape
print("Resulting dataset shape: ", x_train.shape)

## Data overview

In [ ]:
# metrics regarding score distribution
mean_scores = np.sum(y_train * np.arange(1, 11), axis = 1) / (np.sum(y_train, axis = 1) + 1)
mean_scores_rep = np.repeat(mean_scores, 10, axis = 0).reshape(len(y_train), 10)
stds_scores = np.sqrt((np.sum(np.abs(np.arange(1, 11) - mean_scores_rep) ** 2 * y_train, axis = 1)) / (np.sum(y_train, axis = 1)))

In [ ]:
# rating distribution mean over all the dataset
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.title('Histogram of mean scores')
plt.hist(mean_scores, bins = 100)
plt.subplot(1, 2, 2)
plt.title('Histogram of standard deviations')
plt.hist(stds_scores, bins = 100)
plt.show()

In [ ]:
# number of random samples to be plotted
n_rand_samples = 3

# define directory of training images
train_dir_images = os.path.join(workdir, 'images')

# plot n_rand_samples
for i in range(n_rand_samples):

    # pick random training image
    case = os.path.join(train_dir_images, random.choice(os.listdir(train_dir_images)))
    case_filename = os.path.splitext(ntpath.basename(case))[0] + '.jpg'
    print("Filename: ", case_filename)
    
    # plot the score distribution of the image
    plt.figure(figsize=(12, 12))
    plt.subplot(2, 2, 2)

    punt = y_train[x_train == int(os.path.splitext(ntpath.basename(case))[0])][0]
    plt.title('Score distribution')
    plt.bar(np.arange(1,11), height = punt, width = 0.9)

    # open image with opencv and visualize it
    plt.subplot(2, 2, 1)

    image = cv2.cvtColor(cv2.imread(case, 1), cv2.COLOR_BGR2RGB) # cv2 loads an image BGR rather than RGB
    plt.title('Image preview')
    plt.imshow(image)
    plt.show()

**********************

Since the **255530** images are impossible to be loaded in memory, we will use a technique that will let us load images during the training process. Using this technique, we won't need to create a dataset beforehand and then use it to train you network, but we will just have a list of training images available, and images will be extracted **on-the-fly** during training. This strategy allows to save time in the preparation the static dataset as well as memmory, and allows the use of a dynamic generation of batches, where data augmentation can also be applied on-the-fly.

In [ ]:
class BatchCreator(object):

    def __init__(self, x, y, batch_size, augmentation_fn=None):
        '''
        dataset: numpy array containing the Image IDs and the Rating distribution
        '''
        
        # params
        self.imgs_ids        = x                # image IDs
        self.imgs_rates      = y                # image rate distribution
        self.batch_size      = batch_size       # number of patches per batch
        self.augmentation_fn = augmentation_fn  # augmentation function
        
        # batch information
        self.n_samples = len(self.imgs_ids)
        self.n_batches = self.n_samples // self.batch_size
    
        # information print
        print('BatchCreator: {n_samples} patch samples.'.format(n_samples=self.n_samples))
        
    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def __len__(self):
        # length as number of batches
        return self.n_batches

    def next(self):
        # build a mini-batch

        batch_x = np.zeros((self.batch_size, 224, 224, 3), dtype = 'uint8')
        batch_y = np.zeros((self.batch_size, 10)) # one-hot encoding

        # load and return "batch_size" images
        for i in range(self.batch_size):

            # select case (generate random index)
            img_index = np.random.choice(len(self.imgs_ids))

            # obtain the path
            case = os.path.join(train_dir_images, str(self.imgs_ids[img_index]) + '.jpg')

            # load image, change color structure, resize it and store it
            batch_x[i] = cv2.resize(cv2.cvtColor(cv2.imread(case, 1), cv2.COLOR_BGR2RGB), (224, 224))

            # store score distribution
            batch_y[i] = self.imgs_rates[img_index]/np.sum(self.imgs_rates[img_index])

        return batch_x, batch_y

Once the BatchCreator is defined, it will be tested

In [ ]:
training_gen = BatchCreator(x_train, y_train, 16)

# create a batch
for batch_x, batch_y in training_gen:
    print(batch_x.shape)
    print(batch_y.shape)
    break
    
# visualize it
matplotlib.rcParams['figure.figsize'] = (20, 18)
f, axes = plt.subplots(4, 4)
i = 0;
for ax_row in axes:
    for ax in ax_row:
        ax.imshow(batch_x[i])
        ax.axis('off')
        ax.set_title('mean: {}'.format(np.sum(batch_y[i] * np.arange(0, 1)) / (np.sum(batch_y[i]) + 1)))
        i += 1
plt.show()

In [ ]:
class BatchSequence(keras.utils.Sequence):

    def __init__(self, x, y, batch_size):

        # params
        self.imgs_ids   = x          # image IDs
        self.imgs_rates = y          # image rate distribution
        self.batch_size = batch_size # number of patches per batch
        
        # length
        self.n_samples = len(self.imgs_ids)
        self.n_batches = int(np.ceil(self.n_samples / self.batch_size))  # last mini-batch might be shorter
        
        # print some info
        print('PatchSequence: {n_samples} patch samples.'.format(n_samples=self.n_samples))

    def __len__(self):
        # provide length in number of batches
        return self.n_batches
    
    def __getitem__(self, idx):

        # create indexes for samples
        idx1 = idx * self.batch_size
        idx2 = np.min([idx1 + self.batch_size, self.n_samples])
        idxs = np.arange(idx1, idx2)
        
        batch_x = np.zeros((len(idxs), 224, 224, 3), dtype = 'uint8')
        batch_y = np.zeros((len(idxs), 10)) # one-hot encoding

        # load and return "batch_size" images
        for i in range(len(idxs)):

            # obtain the path
            case = os.path.join(train_dir_images, str(self.imgs_ids[idxs[i]]) + '.jpg')

            # load image, change color structure, resize it and store it
            batch_x[i] = cv2.resize(cv2.cvtColor(cv2.imread(case, 1), cv2.COLOR_BGR2RGB), (224, 224))

            # store score distribution
            batch_y[i] = self.imgs_rates[i]/np.sum(self.imgs_rates[idxs[i]])

        return batch_x, batch_y

In [ ]:
validation_gen = BatchSequence(x_train, y_train, 16)

# create a batch
for batch_x, batch_y in validation_gen:
    print(batch_x.shape)
    print(batch_y.shape)
    break
    
# visualize it
matplotlib.rcParams['figure.figsize'] = (20, 18)
f, axes = plt.subplots(4, 4)
i = 0;
for ax_row in axes:
    for ax in ax_row:
        ax.imshow(batch_x[i])
        ax.axis('off')
        ax.set_title('mean: {}'.format(np.sum(batch_y[i] * np.arange(1, 11)) / (np.sum(batch_y[i]) + 1)))
        i += 1
plt.show()

Once we have created the BatchCreator and the BatchSequence, we will split our dataset in trainig and validation.

In [ ]:
# # a subset of samples will be extracted
# perc = 1
# num_samples = int(perc * len(y_train))
# indxs = np.arange(len(y_train))
# sel_idxs = np.random.choice(indxs, num_samples, replace = False)

# # select the samples
# x_train_sub = x_train[sel_idxs]
# y_train_sub = y_train[sel_idxs]

In [ ]:
# split train and validationy_train
x_train, x_validation, y_train, y_validation = train_test_split(x_train, 
                                                                y_train, 
                                                                test_size = 0.2,
                                                                random_state = random_seed)

After splitting the dataset, we will define the loss function `EMD` (Earth Mover’s Distance). Whose equation is the following one:

$$ EMD(p, \widehat{p}) = \left ( \frac{1}{N} \sum_{k=1}^{N} \left | CDF_{p} \left ( k \right ) - CDF_{\widehat{p}}\left ( k \right ) \right |^{r} \right )^{1/r} $$

Where `CDF` is the cummulative distribution, and `r` is fixed to 2 to penalize the Euclidean distance between the
CDFs and allows easier optimization when working with gradient descent.

In [ ]:
def emd_loss(y_true, y_pred):
    '''
    Earth Mover's Distance loss
    '''
    cdf_p    = K.cumsum(y_true, axis = -1)
    cdf_phat = K.cumsum(y_pred, axis = -1)
    loss     = K.mean(K.sqrt(K.mean(K.square(K.abs(cdf_p - cdf_phat)), axis = -1)))
    
    return loss


In [ ]:
# load the pre-trained model
vgg_16 = vgg16.VGG16()

vgg_16.summary()

# remove last layer
vgg_16.layers.pop()
vgg_16.layers.pop()

vgg_16.summary()

# freeze all the layers
for layer in vgg_16.layers:
    layer.trainable = False

# get the output of the model
x = vgg_16.layers[-1].output
    
# add last FC layer
x = Dropout(0.5)(x)
x = Dense(10, activation = 'softmax')(x)

# create the new model
vgg_16 = Model(vgg_16.input, x)

# show summary
vgg_16.summary()

In [ ]:
# learning rat
learning_rate = 1e-3

# optimizer and model compile
optimizer = Adam(lr = learning_rate)
vgg_16.compile(optimizer, loss = emd_loss)

In [ ]:
# training
checkpoint = ModelCheckpoint(os.path.join(result_dir, 'best_model.h5'), 
                             monitor = 'val_loss',
                             verbose = 1,
                             save_weights_only = True,
                             save_best_only = True,
                             mode = 'min')

# batch size and number of epochs
batch_size = 32
epochs = 3

# create the Batches and obtain the generatoros
training_data = BatchCreator(x_train, y_train, batch_size)
validation_data = BatchSequence(x_validation, y_validation, batch_size)

# fit the model
vgg_16.fit_generator(training_data,
                     steps_per_epoch = 2000,
                     epochs = epochs,
                     verbose = 1, 
                     callbacks = [checkpoint],
                     validation_data = validation_data,
                     validation_steps = 300,
                     workers = 8)

## Test model

In [ ]:
# number of random samples to be plotted
n_rand_samples = 3

# obtain n_rand_samples from the validation subset
val_indxs = np.random.randint(0, len(x_validation), n_rand_samples)
val_images_name = x_validation[val_indxs] 
val_labels_true = y_validation[val_indxs]

val_images = []

# load the images into memmory
for image in val_images_name:
    case_filename = str(image) + '.jpg'
    case = os.path.join(workdir, 'images', case_filename)
    val_images.append(cv2.resize(cv2.cvtColor(cv2.imread(case, 1), cv2.COLOR_BGR2RGB), (224, 224)))

val_images = np.array(val_images)
val_labels_pred = vgg_16.predict(val_images)

# plot n_rand_samples
for i in range(n_rand_samples):

    case_filename = str(val_images_name[i]) + '.jpg'
    case = os.path.join(workdir, 'images', case_filename)
    print("Filename: ", case_filename)
    
    plt.figure(figsize=(12, 12))

    # open image with opencv and visualize it
    plt.subplot(2, 3, 1)
    image = cv2.cvtColor(cv2.imread(case, 1), cv2.COLOR_BGR2RGB) # cv2 loads an image BGR rather than RGB
    plt.title('Image preview')
    plt.imshow(image)
    
    # plot the ground truth score distribution of the image
    plt.subplot(2, 3, 2)
    punt_true = val_labels_true[i]
    plt.title('Ground truth score distribution')
    plt.bar(np.arange(1,11), height = punt_true, width = 0.9)
    
    # plot the predicted score distribution of the image
    plt.subplot(2, 3, 3)
    punt_pred = val_labels_pred[i]
    plt.title('Predicted score distribution')
    plt.bar(np.arange(1,11), height = punt_pred, width = 0.9)
    
    plt.show()